In [ ]:
%%bash
pip install astroML
pip install sklearn

In [ ]:
import numpy as np
from jointpdf.jointpdf import JointProbabilityMatrix

In [ ]:
from functools import reduce

def compute_joint(multi_index):
    return reduce(lambda x, y: x*y, [marginal_distributions[combination] for
            combination in zip(range(len(multi_index)), multi_index)])

def compute_joint_from_marginals(marginal_distributions):
    dimension_joint_distribution = [len(dist) for dist in marginal_distributions]
    full_distribution_temp = np.zeros(dimension_joint_distribution)
    full_distribution = np.zeros(dimension_joint_distribution)

    it = np.nditer(full_distribution, flags=['multi_index'], op_flags=[['readwrite']])
    while not it.finished:
        full_distribution[it.multi_index]= compute_joint(it.multi_index)
        it.iternext()

    return full_distribution
    #print(full_distribution)
    


In [ ]:
a = np.zeros((4,3))
b = np.ones((2,2))

a[0:2, 0:2] = b
 
print(a)

In [ ]:
class JointProbabilityMatrixExtended(JointProbabilityMatrix):
    """this class extends the JointProbabilityMatrix by allowing different
    variables to have different statespaces"""
    def __init__(self, numvariables, max_num_states, joint_pdf):
        self.numvariables = numvariables
        self.max_num_states = max_num_states
        self.joint_pdf = joint_pdf
        #super(JointProbabilityMatrix, self).__init__(num_variables, num_states, joint_pdf)
    
    #needed to access methods in parent class
    def create_joint_probabilities_with_uniform_statespace(self):
        self.old_joint_probabilities = self.joint_probabilites 
        max_number_of_states = max(self.joint_probabilites.joint_probabilities.shape)
        temp_joint_probabilities = np.zeros([self.numvariables]*max_number_of_states)
        it = np.nditer(self.joint_probabilites.joint_probabilities, flags=['multi_index'])
        while not it.finished:
            temp_joint_probabilities[it.multi_index] = it.value
            it.iternext()
            
        self.joint_probabilities = temp_joint_probabilities
        self.num_states = max_number_of_states
    
    #rewrite this to work on a nd-array instead!
    def append_determinstic_function(self, func, num_states):
        """append an extra variable to the distribution given by the 
        deterministic function func
        
        params:
            func: a deterministic discrete function or the already defined variables
            num_states: the number of outcomes of the function
        """
        temp1_joint_probability = np.zeros(
            list(self.joint_probabilites.joint_probabilities.shape).append(num_states)
        )
        temp2_joint_probability = np.zeros(
            list(self.joint_probabilites.joint_probabilities.shape).append(num_states)
        )
        it = np.nditer(temp1_joint_probability, flags=['multi_index'])
        while not it.finished:
            arguments = list(it.multi_index)[:-1]
            if func(arguments) == it.multi_index[-1]:
                temp2_joint_probability[it.multi_index] = self.joint_probability[]
                
    

In [ ]:
marginal_distributions = np.array([[0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.8,  0.2]])
pdf = JointProbabilityMatrix(4, 2, compute_joint_from_marginals(marginal_distributions))
pdf_extended = JointProbabilityMatrixExtended(4, 2, compute_joint_from_marginals(marginal_distributions))

In [ ]:
#pdf.append_variables_using_state_transitions_table

#marginal = pdf.marginalize_distribution([0, 1])
#marginal.joint_probabilities.joint_probabilities

marginal = pdf_extended.marginalize_distribution([0, 1])
marginal.joint_probabilities.joint_probabilities

In [ ]:
import itertools

def create_mi_profile(joint_distribution):
    """create a mutual information profile
    
    params:
        joint_distribution: A JointProbabilityMatrix object from the jointpdf package.
        The joint probabilities properties should be of both all the variables and the
        function.
        
    returns: a 1-d nd-array where every value represents the average normalized mutual information
        between subsets of variables (of the size of the index) and the ouput variable. 
    """
    #assuming the function has an one-dimension output
    mi_values = np.zeros(joint_distribution.numvariables)
    function_label = joint_distribution.numvariables-1
    marginal_function = joint_distribution.marginalize_distribution(
        joint_distribution.marginalize_distribution([joint_distribution.numvariables-1])
    )
    for number_of_variables in np.arange(1, joint_distribution.numvariables, 1):
        combinations = itertools.combinations(range(joint_distribution.numvariables-1), 
                                              number_of_variables)
        
        mi_values[number_of_variables] = np.mean(
            [joint_distribution.mutual_information(combination, [function_label]) 
             for combination in combinations]
        )
        
    return mi_values
                
    